# Avaliação Final II - Visualização de dados

## Autor: José Norberto Fagundes Isaias (19202785)

In [265]:
import pandas as pd
import altair as alt
import kagglehub
import os
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

### Fase 1 - preparação dos dados

1. Carregamento dos datasets em .csv e criação de DataFrames contendo apenas as colunas desejadas.

2. Mudança de formato e nome de colunas por conveniência
3. União das duas tabelas através do nome e ano de lançamento dos jogos.

In [266]:
# 1
path = kagglehub.dataset_download("beridzeg45/metacritic-pc-games-reviews") # Necessário o uso do kagglehub devido o tamanho deste dataset. Disponível em https://www.kaggle.com/datasets/beridzeg45/metacritic-pc-games-reviews
raw_reviews = pd.read_csv(os.path.join(path, "metacritic_pc_games.csv")) 
raw_sales = pd.read_csv("Datasets/video games sales.csv") # Disponível em: https://www.kaggle.com/datasets/zahidmughal2343/video-games-sale?select=video+games+sales.csv
raw_imdb = pd.read_csv("Datasets/imdb-videogames.csv") # Disponível em: https://www.kaggle.com/datasets/muhammadadiltalay/imdb-video-games?select=imdb-videogames.csv
raw_meta = pd.read_csv("Datasets/all_video_games.csv") # Disponível em: https://www.kaggle.com/datasets/beridzeg45/video-games

C:\Users\josen\AppData\Local\Temp\ipykernel_24836\1216346433.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_reviews = pd.read_csv(os.path.join(path, "metacritic_pc_games.csv"))


In [267]:
# 1
df_sales = raw_sales[["Name", "Year", "Global_Sales"]].dropna()
df_imdb = raw_imdb[["name", "year", "rating", "Action", "Adventure", "Comedy", "Crime", "Family", "Mystery", "Sci-Fi", "Thriller"]].dropna()
df_meta = raw_meta[["Title", "Release Date", "Genres", "User Score"]].dropna()
df_reviews = raw_reviews[['Game Title', 'Overall Metascore', 'Overall User Rating']].dropna()

# 2
df_imdb.rename(columns={"year": "Year", "name": "Name"}, inplace=True)
df_meta["Release Date"] = pd.to_datetime(df_meta["Release Date"], format='%d/%m/%Y', errors='coerce')
df_meta["Year"] = df_meta["Release Date"].dt.year
df_meta.rename(columns={"Title": "Name"}, inplace=True)

# 3
merged_imdb_sales = pd.merge(df_sales, df_imdb, on=["Name", "Year"], how="inner")
merged = pd.merge(merged_imdb_sales, df_meta, on=["Name", "Year"], how="inner")
merged_imdb_sales.to_csv("merged_imdb_sales.csv", index=False)

#### Fase 1.1 - preparação do DataFrame para o gráfico de diferença entre avaliações de críticos e usuários

1. União entre os DFs do Metacritics com o de reviews de críticos através do nome dos jogos.
2. Passagem do valor dos reviews dos usuários para numérico e depois multiplicação por 10 (O Metascore dos críticos vai até 100, enquanto a nota dos usuários vai até 10)

In [268]:
# 1
df_meta_reviews = pd.merge(df_meta, df_reviews, left_on="Name", right_on="Game Title", how="inner")

# 2
df_meta_reviews['Overall User Rating'] = pd.to_numeric(df_meta_reviews['Overall User Rating'], errors='coerce') * 10

#### Fase 1.2 - preparação do DataFrame para o gráfico de quantidade de jogos por ano

1. Agrupamento dos dados por gênero e ano para calcular a quantidade de vendas por ano para cada gênero.
2. Agrupamento dos dados por ano para calcular a quantidade de jogos lançados de cada gênero ao longo dos anos.
3. Um DF temporário é criado e adicionado a uma lista de DFs.
4. Esta lista é concatenada e os dados dela são usados para criar 3 DFs diferentes: um para a quantidade jogos lançados por ano, outro para as vendas por ano e outro para a razão entre estas duas métricas.
5. Estes 3 DFs são concatenados.


In [269]:
genres = ['Action', 'Adventure', 'Comedy', 'Crime', 'Family', 'Mystery', 'Sci-Fi', 'Thriller']

df_sales_quant_gen = []
for g in genres:
    df_temp = merged_imdb_sales[['Year', 'Global_Sales', g]]
# 1
    df_temp_2 = df_temp.groupby(["Year", g])[['Global_Sales']].sum().reset_index()
    df_temp_2 = df_temp_2[['Global_Sales','Year', g]].loc[df_temp_2[g] == True]
# 2
    df_temp_3 = df_temp.groupby("Year")[[g]].sum().reset_index()

# 3
    df_temp = pd.DataFrame({
        'Year': df_temp_3["Year"],
        'Global_Sales': df_temp_2["Global_Sales"],
        'Quant_gen': df_temp_3[g],
        'Genero': g
    })
# 4
    df_sales_quant_gen.append(df_temp)

df_sales_quant = pd.concat(df_sales_quant_gen, ignore_index=True)

df_line_q = pd.DataFrame({
    'Year': df_sales_quant['Year'],
    'Value': (df_sales_quant["Quant_gen"]),
    'Genero': df_sales_quant['Genero'],
    'Metric': 'Quant. Jogos'
})

df_line_s = pd.DataFrame({
    'Year': df_sales_quant['Year'],
    'Value': (df_sales_quant["Global_Sales"]),
    'Genero': df_sales_quant['Genero'],
    'Metric': 'Vendas'
})

df_line = pd.DataFrame({
    'Year': df_sales_quant['Year'],
    'Value': (df_sales_quant["Global_Sales"] / df_sales_quant["Quant_gen"]),
    'Genero': df_sales_quant['Genero'],
    'Metric': 'Vendas p. Quant. Jogos'
})

# 5
df_long = pd.concat([df_line_q, df_line_s, df_line], ignore_index=True)
df_long = df_long.dropna()

#### Fase 1.3 - preparação do DataFrame para o gráfico de representação de cada gênero nas vendas globais de todos os jogos.

Aqui os dados são colocados em um DF a partir do DF de vendas e quantidade de jogos lançados criado anteriormente.

In [270]:

df_sales_sum = pd.DataFrame({
    'Sales': df_sales_quant['Global_Sales'],
    'Genero': df_sales_quant['Genero'],
    'Year': df_sales_quant['Year']
})

#### Fase 1.4 - criação do DataFrame para o gráfico das notas médias por gênero

In [271]:
df_ratings = merged.groupby("Genres")["User Score"].mean().reset_index()

### Fase 2 - configuração dos gráficos

1. Criação de parâmetro para a barra deslizante que controla o ano de início do gráfico de vendas por gênero
2. Criação do dropdown que permite a seleção dos dados a serem mostrados no gráfico de vendas/jogos lançados por gênero ao longo dos anos.
3. Configuração das seleções por legenda dos gráficos de notas por gênero e de diferença de notas de críticos e de usuários.

In [272]:
# 1
year_min = int(df_long['Year'].min())
year_max = int(df_long['Year'].max())
year_param = alt.param(name='Ano', bind=alt.binding_range(min=year_min, max=year_max, step=1), value=2000)

# 2
type_dropdown = alt.param(
    name='Metrica',
    bind=alt.binding_select(options=df_long['Metric'].unique().tolist()),
    value='Vendas p. Quant. Jogos'   # valor inicial
)

# 3
selection_legend = alt.selection_point(fields=['Genres'], bind='legend')

#### Fase 2.1 - instanciação do gráfico de participação de vendas por gênero no total global de vendas

Aqui é instanciado um gráfico de rosquinha com a fatia de cada gênero nas vendas globais de todos os jogos. O intervalo de tempo observado pode ser controlado através de uma barra deslizante.

In [273]:
graph_share = alt.Chart(df_sales_sum).mark_arc(innerRadius = 75).encode(
    theta=alt.Theta(field="Sales", type="quantitative", title="Média de Avaliação"),
    color=alt.Color(field="Genero", type="nominal", title="Gênero"),
    tooltip=[alt.Tooltip('Genero:N', title='Gênero'), alt.Tooltip('Sales:Q', title='Participação no total de vendas')]
).add_params(year_param).transform_filter(alt.datum.Year >= year_param)

#### Fase 2.2 - instanciação do gráfico vendas/quantidade de jogos lançados por gênero ao longos dos anos.

Aqui é instanciado um gráfico de linha de tempo ao longo dos anos que pode mostrar a quantidade absoluta de vendas por gênero, a quantidade absoluta de jogos lançados por gênero e uma razão entre estas duas quantidades. Os anos observados podem ser controlados por uma barra deslizante e o tipo de dado no eixo y pode ser mudado por um combobox.

In [274]:
graph_sales = alt.Chart(df_long).mark_line(point=True).encode(
    x=alt.X('Year:O', title='Ano'),
    y=alt.Y('Value:Q', title='Vendas globais (M) / quantidade de jogos', scale=alt.Scale(zero=False)),
    color=alt.Color('Genero:N', title='Gênero'),
    tooltip=[alt.Tooltip('Year:O', title='Ano'), alt.Tooltip('Value:Q', title='Vendas globais (M) / quantidade de jogos'), alt.Tooltip('Genero:N', title='Gênero')]
).properties(
    title='Relação entre Vendas Globais e Quantidade de Jogos por Gênero ao Longo dos Anos',
    width=800, height=350
).transform_filter(
    (alt.datum.Year >= year_param) &
    (alt.datum.Metric == type_dropdown)
).add_params(year_param, type_dropdown)

#### Fase 2.3 - instanciação do gráfico de notas dos usuários por gênero dos jogos

Aqui é instanciado um gráfico de barras que mostra a média das notas dos jogos por gênero da dataset do Metacritic. Também é criado uma linha indicadora do valor médio de todas as notas para comparação com cada gênero. Os gêneros a serem comparados podem ser escolhidos apertando shift e clicando nos gêneros na legenda do lado direito do gráfico.

In [275]:
graph_rating = alt.Chart(df_ratings).mark_bar().encode(
    x=alt.X("Genres:O", title="Gênero"),
    y=alt.Y("User Score:Q", title="Média de Avaliação"),
    size=alt.condition(selection_legend, alt.value(10), alt.value(0)),
    color=alt.Color("Genres:N", title = "Gêneros", legend=alt.Legend(columns=2, labelLimit=100,  symbolLimit=100)),
    tooltip=[alt.Tooltip('Genres:O', title='Gênero'), alt.Tooltip('User Score:Q', title='Média de Avaliação')]
).properties(
    width = 800,
    title = "Nota média por gênero de jogo"
).add_params(selection_legend)

rule = alt.Chart(df_ratings).mark_rule(color='red').encode(
    y='mean(User Score):Q',
    tooltip=alt.Tooltip('mean(User Score):Q', title='Média Geral')
)

#### Fase 2.4 - instanciação do gráfico de comparação entre as notas dos críticos em relação a dos usuários. 

Aqui é instanciado um gráfico de barras sobrepostas. Neste gráfico, as barras pretas e opacas reprensentam as notas médias dos críticos do Metacritic (Metascore), enquanto as barras coloridas, sobrepostas as barras pretas e com transparência, representam a nota média dada pelos usuários para os jogos de cada gênero. Os gêneros a serem comparados podem ser escolhidos apertando shift e clicando nos gêneros na legenda do lado direito do gráfico.

In [276]:
graph_reviews_critics = alt.Chart(df_meta_reviews).mark_bar(color='black').encode(
    x = alt.X("Genres:O", title="Gênero"),
    y = alt.Y("mean(Overall Metascore):Q", title="Média de Avaliação"),
    size=alt.condition(selection_legend, alt.value(5), alt.value(0)),
    tooltip=[alt.Tooltip('Genres:O', title='Gênero'), alt.Tooltip("mean(Overall Metascore):Q", title='Média de Avaliação')]
).properties(
    width = 800,
    title = "Notas dos críticos comparadas as dos usuários"
).add_params(selection_legend)

graph_reviews_users = alt.Chart(df_meta_reviews).mark_bar(opacity=0.7).encode(
    x = alt.X("Genres:O", title="Gênero"),
    y = alt.Y("mean(Overall User Rating):Q", title="Média de Avaliação"),
    size=alt.condition(selection_legend, alt.value(5), alt.value(0)),
    color=alt.Color("Genres:N", title = "Gêneros", legend=alt.Legend(columns=3, labelLimit=120,  symbolLimit=120)),
    tooltip=[alt.Tooltip('Genres:O', title='Gênero'), alt.Tooltip("mean(Overall User Rating):Q", title='Média de Avaliação')]
).properties(
    width = 800,
    title = "Notas dos críticos comparadas as dos usuários"
).add_params(selection_legend)

graph_reviews = alt.layer(graph_reviews_critics, graph_reviews_users).properties(
    title='Diferença de notas dos críticos (barras pretas) vs. notas de usuarios por gênero',
).add_params(selection_legend)

d:\Documentos\Visualizacao de dados\vis_dados\Lib\site-packages\altair\vegalite\v6\api.py:4899: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  return copy.copy()


### Fase 3 - chamada dos gráficos

Aqui são desenhados os gráficos criados anteriormente.

In [277]:
graph_share.interactive()

alt.Chart(...)

In [278]:
graph_sales.interactive()

alt.Chart(...)

In [279]:
(graph_rating + rule).interactive()

alt.LayerChart(...)

In [280]:
graph_reviews.interactive()

alt.LayerChart(...)